**version 0.2**

K-fold cross-validation is added

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy

import torch
from sklearn.model_selection import KFold 
torch.manual_seed(3)

In [2]:
%matplotlib inline

In [3]:
from torch.utils.data import TensorDataset, DataLoader

def load_TCGA_data(type_cancer):
  rawData = pd.read_csv('./TCGA_10n/TCGA_' + type_cancer + '_feature.txt', 
                      sep='\t',
                      names = ['obs'] + [str(1 + n) for n in range(31)])
  rawData.loc[:,'1':] = (rawData.loc[:,'1':]-rawData.loc[:,'1':].mean())/rawData.loc[:,'1':].std()

  rawLabel= pd.read_csv('./TCGA_10n/TCGA_' + type_cancer + '_label.txt', 
                        names=['label'])
  num_obs = len(rawLabel['label'])
  num_instance_variable = len(rawData.columns) - 1
  # num_train_data = int(np.ceil(num_obs * train_prop))
  # num_valid_data = int(num_obs - num_train_data)
  uelem, cnt = np.unique(rawLabel['label'], return_counts = True)
  print("# of instance-level variables=", num_instance_variable)
  print("# of observations=", num_obs)
  print("0-bag : 1-bag =", int(cnt[uelem==0]), ":", int(cnt[uelem==1]))

  ###
  uelem, cnt = np.unique(rawData['obs'], return_counts = True)
  # print(uelem, cnt)
  # print(len(uelem))

  data_list = []
  for i in uelem:
      # index = i + 1
      data_list.append(rawData.loc[rawData['obs'] == i,'1':].to_numpy().tolist())

  ###
  data_list_max_length = []
  src_key_mask = []
          
  for i in range(num_obs):
          
      if len(data_list[i]) >= max_length:
          data_list_max_length.append(data_list[i][:max_length])
          src_key_mask.append(np.zeros(max_length).tolist())
          
      else:
          temp_pad = np.zeros((max_length - len(data_list[i]), num_instance_variable)).tolist()
          temp_ins = data_list[i]
          temp_ins = temp_ins + temp_pad
          
          temp_mask = np.zeros((max_length))
          temp_mask[len(data_list[i]):] = -10000000000000000
          data_list_max_length.append(temp_ins)
          src_key_mask.append(temp_mask.tolist())

  ###
  data_array_max_length = np.array(data_list_max_length)
  mask_array_max_length = np.array(src_key_mask)
  label_array = rawLabel.to_numpy()

  ###
  data = torch.tensor(data_array_max_length, dtype=torch.float32)
  mask = torch.tensor(mask_array_max_length, dtype=torch.float32)
  target = torch.tensor(label_array, dtype=torch.float32)

  dataset_all = TensorDataset(data, mask, target)
  return dataset_all, [num_obs, num_instance_variable]

In [4]:
# data_list = []

# for i in range(num_obs):
#     index = i + 1
#     data_list.append(rawData.loc[rawData['obs'] == index,'1':].to_numpy().tolist())

In [5]:
# _, counts = np.unique(rawData['obs'], return_counts=True)

In [6]:
# data_list_max_length = []
# src_key_mask = []
        
# for i in range(num_obs):
        
#     if len(data_list[i]) >= max_length:
#         data_list_max_length.append(data_list[i][:max_length])
#         src_key_mask.append(np.zeros(max_length).tolist())
        
#     else:
#         temp_pad = np.zeros((max_length - len(data_list[i]), num_instance_variable)).tolist()
#         temp_ins = data_list[i]
#         temp_ins = temp_ins + temp_pad
        
#         temp_mask = np.zeros((max_length))
#         temp_mask[len(data_list[i]):] = 1
        
#         data_list_max_length.append(temp_ins)
#         src_key_mask.append(temp_mask.tolist())

In [7]:
# data_array_max_length = np.array(data_list_max_length)
# mask_array_max_length = np.array(src_key_mask)
# label_array = rawLabel.to_numpy()

In [8]:
# from torch.utils.data import  TensorDataset, DataLoader

# data = torch.tensor(data_array_max_length, dtype=torch.float32)
# mask = torch.tensor(mask_array_max_length, dtype=torch.bool)
# target = torch.tensor(label_array, dtype=torch.float32)

# dataset_all = TensorDataset(data, mask, target)
# train_data, valid_data = torch.utils.data.random_split(dataset_all, [num_train_data, num_valid_data])

# train_loader = DataLoader(train_data, batch_size=num_train_data, shuffle=True)
# valid_loader = DataLoader(valid_data, batch_size=num_valid_data)

In [10]:
import math

import torch.nn as nn
import torch.nn.functional as F
#from torchsummary import summary

from sparsemax import Sparsemax
from sparsemaxth import Sparsemaxth

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
thre = 0
class Att_DMIL(nn.Module):

    def __init__(self, ninp, nhead, nhid, nlayers, dropout=0.5):
        super(Att_DMIL, self).__init__()
        self.model_type = 'AttDMIL'
                
        self.ninp = ninp
            
        self.waint = nn.Linear(ninp, ninp, bias=False)
        self.waout = nn.Linear(ninp, 1, bias=False)
        self.softmax   = nn.Softmax(dim=1)
        self.sparsemax = Sparsemax(dim=1)
        self.sparsemaxth = Sparsemaxth(dim=1)
        
        self.fclayer1 = nn.Linear(ninp, ninp, bias=True)
        self.drout1   = nn.Dropout(dropout)
        self.fclayer2 = nn.Linear(ninp, ninp, bias=True)
        self.drout2   = nn.Dropout(dropout)
        self.fclayer3 = nn.Linear(ninp, ninp, bias=True)
        self.drout3   = nn.Dropout(dropout)
        self.fclayer4 = nn.Linear(ninp, ninp, bias=True)
        self.drout4   = nn.Dropout(dropout)
        self.fclayer5 = nn.Linear(ninp, ninp, bias=True)
        self.drout5   = nn.Dropout(dropout)
        
        self.bn1      = nn.BatchNorm1d(ninp)
        
        self.decoder_f = nn.Linear(ninp, ninp)
        self.decoder_s = nn.Linear(ninp, 1)

    def forward(self, src, mask):
        
        output = torch.relu(self.drout1(self.fclayer1(src)))
        identity = output
        output = torch.relu(self.drout2(self.fclayer2(output)))
        output = torch.relu(self.drout3(self.fclayer3(output)))
        output = output + identity
        #identity = output
        output = torch.relu(self.drout4(self.fclayer4(output)))
        #output = torch.relu(self.drout5(self.fclayer5(output)))
        #output = output + identity
                
        #attw = torch.squeeze(self.waout(torch.relu(self.waint(output))))
        attw = torch.squeeze(self.waout(output))
        attw = torch.transpose(attw, 0, 1)
        attw = attw + mask
        #attw = self.softmax(attw)
        attw = self.sparsemax(attw)
        
        output = torch.transpose(output, 0, 1)
        output = torch.bmm(torch.unsqueeze(attw, 1), output)
        output = torch.squeeze(output)
        
        output = self.decoder_f(output)
        
        output = torch.relu(self.bn1(output))
        
        output = self.decoder_s(output)
        return output, attw

In [11]:
###
epochs = 100 # The number of epochs
nlayers = 10 # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 3 # the number of heads in the multiheadattention models
dropout = 0.3 # the dropout value


max_length = 90
batch_size = 50


K = 10
random_state = 1

###
# type_cancers = ['BRCA', 'KIRC', 'LUAD', 'LUSC', 'ESCA', 'STAD']
# type_cancers = ['STAD']
type_cancers = ["BRCA", "KIRC","LUAD","LUSC","DLBC", "SKCM", "ESCA","OV","THYM", "STAD"]
## "BLCA","HNSC","LIHC"

## The following types are not suitable for data-splitting, thus skipped
# 'SKCM': 0:1 = 1:423
# 'DLBC': 0:1 = 1:48
# 'OV': 0:1 = 1:407
# 'THYM': 2:120

result_all = list()
attn_all   = []
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
valid_index_all = []
for type_cancer in type_cancers:
  print("Cancer type=", type_cancer)
  dataset_all, param = load_TCGA_data(type_cancer)
  
  ninp = param[1]
  nhid = param[1] # the dimension of the feedforward network model in nn.TransformerEncoder
  
  valid_index_list = []
  Kfold = KFold(n_splits = K, shuffle = True, random_state = 1)
  for train_index, valid_index in Kfold.split(dataset_all):
    valid_index_list = valid_index_list + valid_index.tolist()
    # idx_fold += 1
    # Split data
    # print("Train:", train_index, "Validation:",valid_index)
    # x_train_fold, x_valid_fold = x_train[train_index], x_train[valid_index] 
    # y_train_fold, y_valid_fold = y_train[train_index], y_train[valid_index]
    train_data = torch.utils.data.dataset.Subset(dataset_all, train_index)
    valid_data = torch.utils.data.dataset.Subset(dataset_all, valid_index)
    # train_data, valid_data = torch.utils.data.random_split(dataset_all, [num_train_data, num_valid_data])
    
    train_loader = DataLoader(train_data, batch_size=len(train_index), shuffle=True)
    valid_loader = DataLoader(valid_data, batch_size=len(valid_index))
  
    model = Att_DMIL(ninp, nhead, nhid, nlayers, dropout).to(device)

    ###
    criterion = nn.BCELoss()
    lr =10 # learning rate
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.99)

    import time
    from sklearn.metrics import roc_auc_score

    m = nn.Sigmoid()

    def train():
        model.train() # Turn on the train mode
        total_loss = 0.
        start_time = time.time()
        batch = 0
        for data, mask, targets in train_loader:
            data, mask, targets = torch.transpose(data, 1, 0).to(device), mask.to(device), targets.to(device)
            optimizer.zero_grad()
            
            output, _ = model(data, mask)
            loss = criterion(m(output), targets)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
            optimizer.step()
            
            batch += 1
            total_loss += loss.item()
            log_interval = 1
            if batch % log_interval == 0 and batch > 0:
                cur_loss = total_loss / log_interval
                elapsed = time.time() - start_time
    #             print('| epoch {:3d} | {:5d}/{:5d} batches | '
    #                   'lr {:02.2f} | ms/batch {:5.2f} | '
    #                   'loss {:5.2f}'.format(
    #                     epoch, batch, len(train_loader.dataset) // batch_size, scheduler.get_last_lr()[0],
    #                     elapsed * 1000 / log_interval,
    #                     cur_loss))
                total_loss = 0
                start_time = time.time()

    def evaluate(eval_model, eval_loader):
        eval_model.eval() # Turn on the evaluation mode
        total_loss = 0.
        #total_attn = []
                
        with torch.no_grad():
            correct = 0
            for data, mask, targets in eval_loader:
                data, mask, targets = torch.transpose(data, 1, 0).to(device), mask.to(device), targets.to(device)
                
                output, attn = eval_model(data, mask)
                loss = criterion(m(output), targets)
                total_loss += len(data[0]) * loss.item()
                #total_attn.append(attn)
                #torch.set_printoptions(threshold=10_000)
                #print(attn)
                
                # seongoh
                auc = roc_auc_score(targets.cpu(), m(output.cpu()))
                y_pred_tag = torch.round(torch.sigmoid(output))
                correct_results_sum = (y_pred_tag == targets).sum().float()
                correct += correct_results_sum
                acc = correct.cpu()/len(eval_loader.dataset)
                
        return total_loss / (len(eval_loader.dataset) - 1), float(acc.numpy()), auc, attn, mask
    ###
    criterion = nn.BCELoss()
    lr =10 # learning rate
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.99)
    m = nn.Sigmoid()

    ###
    best_val_loss = float("inf")
    best_model = None

    for epoch in range(1, epochs + 1):
        epoch_start_time = time.time()
        train()
        val_loss, _, _, _, _ = evaluate(model, valid_loader)
#         if epoch % 50 == 0:
#           print('-' * 89)
#           print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} |'
#                 .format(epoch, (time.time() - epoch_start_time),val_loss))                                      
#           print('-' * 89)

        if val_loss < best_val_loss:
            # print('best model!!!!')
            best_val_loss = val_loss
            best_model = copy.deepcopy(model)

        scheduler.step()

    ###
    # train_msr = evaluate(best_model, train_loader)
    test_msr = evaluate(best_model, valid_loader)
    test_attn = test_msr[3]
    test_mask = test_msr[4]
    test_msr = test_msr[0:3]
    # print(train_msr)
    print(test_msr)
    test_attn = test_attn + test_mask
    res_test = list(test_msr)
    res_test.insert(0, type_cancer)
    print(res_test)
    result_all.append(res_test)
    attn_all.append(test_attn)
    
  valid_index_all.append(valid_index_list)

Cancer type= BRCA
# of instance-level variables= 31
# of observations= 225
0-bag : 1-bag = 202 : 23
(0.19765328074043448, 0.95652174949646, 0.8636363636363636)
['BRCA', 0.19765328074043448, 0.95652174949646, 0.8636363636363636]
(0.40983284603465686, 0.8260869383811951, 0.8157894736842106)
['BRCA', 0.40983284603465686, 0.8260869383811951, 0.8157894736842106]
(0.1557067876512354, 0.95652174949646, 0.9545454545454546)
['BRCA', 0.1557067876512354, 0.95652174949646, 0.9545454545454546]
(0.48119337585839356, 0.8260869383811951, 0.5263157894736842)
['BRCA', 0.48119337585839356, 0.8260869383811951, 0.5263157894736842]
(0.1937871209599755, 0.95652174949646, 0.7727272727272727)
['BRCA', 0.1937871209599755, 0.95652174949646, 0.7727272727272727]
(0.3670918345451355, 0.8636363744735718, 0.7719298245614035)
['BRCA', 0.3670918345451355, 0.8636363744735718, 0.7719298245614035]
(0.32707248415265766, 0.9090909361839294, 0.575)
['BRCA', 0.32707248415265766, 0.9090909361839294, 0.575]
(0.23049454745792208

In [ ]:
from prettytable import PrettyTable

dat = np.asarray(result_all)[:,np.r_[0,3]]
dat[:,1] = np.round(100 * np.asarray(dat[:,1], dtype=np.float64), 1)
dat = np.insert(dat, 2, np.repeat([61.8, 68.1, 62.1, 56.8, 78.1, 66.5, 83.1, 82.7, 84.7, 87.3], K), axis=1)
# print(dat)
df = pd.DataFrame(dat, columns = ["Cancer", "AUC", "AUC_max_others"])
df['AUC'] = pd.to_numeric(df['AUC'])
df['AUC_max_others'] = pd.to_numeric(df['AUC_max_others'])
# print(df)
# df.info()
df.groupby('Cancer').mean()
# x = PrettyTable(["Cancer", "AUC", "AUC_max_others"])
# for row in dat:
#     x.add_row(row)
# print(x)

In [ ]:
for i in range(len(type_cancers)):
    idx = np.r_[(i * K):((i+1) * K)]
    lst_attn_subset = [attn_all[i] for i in range(i * K, (i+1) * K)]

    arr_attn = torch.Tensor(len(lst_attn_subset), 2, 2)
    arr_attn = torch.cat(lst_attn_subset, dim=0)
    arr_attn = arr_attn.cpu()
    # open(arr_attn)
    # f = open('/TCGA_attn_' + type_cancers[i] + ".csv", 'w')
    # writer = csv.writer(f)
    # writer.writerow(arr_attn)
    # np.savetxt('/content/drive/MyDrive/TCGA_attn_' + type_cancers[i] + ".csv", 
    #         arr_attn, delimiter=',')
    arr_attn = np.array(arr_attn)
    arr_attn[arr_attn == -1e+16] = np.nan
    pd.DataFrame(arr_attn).to_csv('./Results_b/TCGA_attn_' + type_cancers[i] + "_" + str(max_length) + ".csv", na_rep='NA')

In [12]:
import pandas
df = pandas.DataFrame(valid_index_all)
df.to_csv("./index_file_im.csv", sep=',',index=False)